In [43]:
import pandas
import re

from lab_v2.xlsx_creation import XlsxWorkbook, XlsxSheet

In [44]:
DRAW_T03 = 'cache/draw/draw-T0.3.json'
DRAW_T05 = 'cache/draw/draw-T0.5.json'
DRAW_T07 = 'cache/draw/draw-T0.7.json'

CSQA_T03 = 'cache/csqa/csqa-T0.3.json'
CSQA_T05 = 'cache/csqa/csqa-T0.5.json'
CSQA_T07 = 'cache/csqa/csqa-T0.7.json'

LAST_LETTERS_T03 = 'cache/last_letters/last_letters-T0.3.json'
LAST_LETTERS_T05 = 'cache/last_letters/last_letters-T0.5.json'
LAST_LETTERS_T07 = 'cache/last_letters/last_letters-T0.7.json'

In [45]:
for cache_file_path in [
    'cache/draw/draw-T0.3',
    'cache/draw/draw-T0.5',
    'cache/draw/draw-T0.7',
    'cache/csqa/csqa-T0.3',
    'cache/csqa/csqa-T0.5',
    'cache/csqa/csqa-T0.7',
    'cache/last_letters/last_letters-T0.3',
    'cache/last_letters/last_letters-T0.5',
    'cache/last_letters/last_letters-T0.7',
]:
    xlsx_file_path = 'xlsx/' + '/'.join(cache_file_path.split('/')[1:]) + '.xlsx'
    print(xlsx_file_path)
    data = pandas.read_json(cache_file_path + '.json', lines=True)
    data = data.sort_values(by='clf', ascending=False)
    data['name'] = data['clf'].apply(lambda row : re.sub(r'Classifier\(.*', 'Classifier', row, flags=re.DOTALL))
    data['layer_count'] = data['clf'].apply(lambda row : row.count('Dense') - 1)
    data['name'] = data['name'].apply(lambda row : re.sub(r'\(|\)|\'', '', row, flags=re.DOTALL))
    data['name'] = data.apply(lambda row : row['name'] if row['layer_count'] == -1 else 'Tensorflow ' + str(row['layer_count']) + ' layers', axis=1)
    
    data['R squared'] = data['r2']
    data['Median Absolute Error'] = data['neg_median_absolute_error'].apply(lambda row : -row)
    data['Mean Absolute Error'] = data['neg_mean_absolute_error'].apply(lambda row : -row)
    data['Max Error'] = data['max_error'].apply(lambda row : -row)

    COLUMNS = ['name', 
    'R squared', 
    'Median Absolute Error', 
    'Mean Absolute Error',
    'Max Error']
    none_df = data[data['sampler'] == 'None'][COLUMNS]
    undersampled_df = data[data['sampler'] == 'RandomUnderSampler(random_state=42)'][COLUMNS]
    oversampled_df = data[data['sampler'] == 'RandomOverSampler(random_state=42)'][COLUMNS]

    workbook = XlsxWorkbook(xlsx_file_path)
    format = workbook.workbook.add_format()
    format.set_text_wrap()

    for df, name in [
        (none_df, 'None'),
        (undersampled_df, 'Random_Undersampling'),
        (oversampled_df, 'Random_Oversampling')
    ]:
        none: XlsxSheet = workbook.add_sheet(name, dataframe=df)
        workbook.workbook.get_worksheet_by_name(name).set_column(1, 6, 15)
        none_chart = none.add_column_chart('Model Performance', 'H1')
        none_chart.set_size({'width': 720, 'height': 400})
        index = 0
        for _, row in none_df.iterrows():
            index += 1
            none_chart.add_series({
                'name': f'={name}!$B${index + 1}',
                'categories': f'={name}!$D$1:$G$1',
                'values': f'={name}!$D${index + 1}:$G${index + 1}',
            })
        none_chart = none.add_column_chart('Model Performance', 'H22')
        none_chart.set_size({'width': 720, 'height': 400})
        index = 0
        for _, row in none_df.iterrows():
            index += 1
            none_chart.add_series({
                'name': f'={name}!$B${index + 1}',
                'categories': f'={name}!$C$1:$C$1',
                'values': f'={name}!$C${index + 1}:$C${index + 1}',
            })
    workbook.close()

xlsx/draw/draw-T0.3.xlsx
xlsx/draw/draw-T0.5.xlsx
xlsx/draw/draw-T0.7.xlsx
xlsx/csqa/csqa-T0.3.xlsx
xlsx/csqa/csqa-T0.5.xlsx
xlsx/csqa/csqa-T0.7.xlsx
xlsx/last_letters/last_letters-T0.3.xlsx
xlsx/last_letters/last_letters-T0.5.xlsx
xlsx/last_letters/last_letters-T0.7.xlsx
